<a id='top'></a>

# CSCI 3022: Intro to Data Science - Spring 2019 Practicum 2
***

This practicum is due on Canvas by **11:59 PM on Friday May 3**. Your solutions to theoretical questions should be done in Markdown/MathJax directly below the associated question.  Your solutions to computational questions should include any specified Python code and results as well as written commentary on your conclusions.  

**Here are the rules:** 

1. All work, code and analysis, must be your own. 
1. You may use your course notes, posted lecture slides, textbooks, in-class notebooks, and homework solutions as resources.  You may also search online for answers to general knowledge questions like the form of a probability distribution function or how to perform a particular operation in Python/Pandas. 
1. This is meant to be like a coding portion of your midterm exam. So, the instructional team will be much less helpful than we typically are with homework. For example, we will not check answers, help debug your code, and so on.
1. If something is left open-ended, it is because we want to see how you approach the kinds of problems you will encounter in the wild, where it will not always be clear what sort of tests/methods should be applied. Feel free to ask clarifying questions though.
2. You may **NOT** post to message boards or other online resources asking for help.
3. You may **NOT** copy-paste solutions *from anywhere*.
4. You may **NOT** collaborate with classmates or anyone else.
5. In short, **your work must be your own**. It really is that simple.

Violation of the above rules will result in an immediate academic sanction (*at the very least*, you will receive a 0 on this practicum or an F in the course, depending on severity), and a trip to the Honor Code Council.

**By submitting this assignment, you agree to abide by the rules given above.**

***

**Name**:  James Gallmeister

***


**NOTES**: 

- You may not use late days on the practicums nor can you drop your practicum grades. 
- If you have a question for us, post it as a **PRIVATE** message on Piazza.  If we decide that the question is appropriate for the entire class, then we will add it to a Practicum clarifications thread. 
- Do **NOT** load or use any Python packages that are not available in Anaconda 3.6. 
- Some problems with code may be autograded.  If we provide a function API **do not** change it.  If we do not provide a function API then you're free to structure your code however you like. 
- Submit only this Jupyter notebook to Canvas.  Do not compress it using tar, rar, zip, etc. 
- This should go without saying, but... For any question that asks you to calculate something, you **must show all work to receive credit**. Sparse or nonexistent work will receive sparse or nonexistent credit.

---
**Shortcuts:**  [Problem 1](#p1) | [Problem 2](#p2) | [Bottom](#bot)

---

In [1]:
# import whatever packages you think you will need here!
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
%matplotlib inline

<br>

---
<a id='p1'></a>
[Back to top](#top)

### [30 points] Problem 1:  What's the best Oreo flavor?

> <img src="https://media2.s-nbcnews.com/j/newscms/2015_52/1351976/151223-watermelon-oreos-flickr-yh-1234p_a4c4ba1016261e5d799a789666948a6e.fit-760w.jpg" style="width: 400px;"/>
**Figure 1.** Watermelon Oreos. These are undeniably the absolute _worst_ Oreo flavor, but not a part of our data set. If they were, the problem would be too easy.

You're skipping down the road one day, singing a merry tune without a care in the world. Every so often you hop and click your heels together. After one such hop, though, you stumble and nearly trip. Under your feet is a fun data set!

In particular, this rascally data set includes ratings from 5 different individuals for 12 different types of Oreo cookie. Because some of the raters are allergic to disgusting flavors of Oreo, not every taster tasted every type of cookie. More information about the data set can be found [here](https://www.kaggle.com/rtatman/oreo-flavors-tastetest-ratings).

As you examine the data, a booming voice from the sky instructs you to analyze the data to answer two key questions:
1. is there some difference in mean rating given to different flavors of Oreo? and
2. which flavor (out of those in the data set) is the best?

**Part A:** Load the data set as a Pandas DataFrame. Then, process it as you see fit. For the analysis, you might create a new DataFrame, or other kinds of Python objects to use the data. Do what feels right, and include an **explanation** in Markdown of how you are processing the data. You **may not** alter the original data file.

In [5]:
df = pd.read_csv('oreo_rankings.csv', na_values='-')
df = df.drop('notes_and_discussion', axis=1)

dfA = df.T
dfA.columns = dfA.iloc[0]
dfA = dfA[1:]
flavs = {}

for x in range(len(df.index)):
    key = list(df.iloc[x])[0]
    temp = list(df.iloc[x])[1:7]
    temp = np.array([_ for _ in temp if str(_) != 'nan'])
    flavs[key] = temp

dfA.head(8)

oreo_flavor,pb&j,mega stuf,lemon,chocolate,birthday cake,double stuf,red velvet,Dunkin' Donuts Mocha,mini,"thins, coconut",mint,cinnamon bun
rater_n,1.5,2.5,4.5,3.5,2.5,3.5,4,2.5,3,3.5,3.5,NaN
rater_c,4,3,3.5,3,5,4,3,1,4,4.5,4,4.5
rater_r,2,4,1.5,3.5,2.5,3,4.5,5,4,3,4.5,4.5
rater_k,NaN,NaN,NaN,3,4,3,4,5,NaN,4.5,4,4.5
rater_a,NaN,1.5,3.5,4,3,3.5,2.5,5,4.25,4.5,4.25,4
average_across_raters,2.5,2.75,3.25,3.4,3.4,3.4,3.6,3.7,3.8125,4,4.05,4.375


**Answer A**
<br>I will be creating a new dataframe that puts the raters as row indices and their respective ratings as features for the row. This is so that I have a dataframe that matches the same structure as the one we worked with in the in-class notebook. I will be dropping the "notes_and_discussion" column as that is useless in my mind and won't serve any purpose in our hypothesis testing. I also created a dictionary with each flavor of cookies as the keys and the raters values in lists so that when doing calculations in regards to oreo cookies I don't have to worry about non-values.

**Part B:** In the remainder of this problem, you will perform some hypothesis tests to examine whether these data suggest there are significant differences in the mean ratings of the different Oreo flavors. Pick a level of significance for these experiments, and explain how you decided to use that significance level. "Because we used it a lot in class" is ***not*** a good reason.

**Answer B**
<br>I have decided to choose $\alpha = 0.1$ as my significance level for this problem because our dataset does not contain many values and I believe that if there is to be a significant difference it will not show up at the 1% or even 5% significance level due to the small number of data points.

**Part C:** Perform an **Analysis of Variance hypothesis test** in order to determine if there is evidence that there is _some_ difference among the mean ratings given to these 12 Oreo flavors. Clearly state your null and alternative hypothesis, and use the significance level identified in **Part B**. You must show **all** calculations **by hand** (and may of course use Python as a calculator, and to compute values from a distribution using the appropriate percent-point-function (ppf) or cumulative distribution function (cdf)).

In addition to showing the code for your calculations, make comments **in Markdown** explaining what you are doing.

**Answer C**
<br>For this problem I will be using the following hypothesis' and significance level
$$
H_0 : \mu_1 = \mu_2 = ... = \mu_i \\
H_1 : \mu_i \ne \mu_j \text{ for some i,j} \\
\alpha = 0.1
$$

In [6]:
#n - total number of data points, I - number of groups, grandmn - grand mean
n = 54
I = len(dfA.columns)
grandmn = np.mean(dfA.dropna().values)
print('##### Calculations by Hand #####')
print('The grand mean of the data is: {0:.3f}'.format(grandmn))

#Computing the SSB and SSW => SST
ssb = 0
ssw = 0
for key in flavs.keys():
    ssb += (len(flavs[key]) - 1) * ((flavs[key][-1] - grandmn)**2)
    ssw += sum((flavs[key][:-1] - flavs[key][-1]) ** 2)

ssb_df = I-1
ssw_df = n-I
print('The between groups sum of squares is: {0:.3f} with SSB_df: {1}'.format(ssb, ssb_df))
print('The within groups sum of squares is: {0:.3f} with SSW_df: {1}'.format(ssw, ssw_df))
print('The total sum of squares is: {0:.3f}'.format(ssb+ssw))

f_stat = (ssb/ssb_df)/(ssw/ssw_df)
pval = 1 - stats.f.cdf(f_stat, ssb_df, ssw_df)
print('Our f-statistic is: {0:.3f}'.format(f_stat))
print('Our p-value is: {0:.3f}'.format(pval))

##### Calculations by Hand #####
The grand mean of the data is: 3.548
The between groups sum of squares is: 11.952 with SSB_df: 11
The within groups sum of squares is: 37.759 with SSW_df: 42
The total sum of squares is: 49.711
Our f-statistic is: 1.209
Our p-value is: 0.311


Because the calculated $\text{p-value} = 0.311$ is greater than the significance level, we conclude that at the 10% significance level there is not enough evidence to reject the null hypothesis. 

**Analysis for C**
<br>For the above calculations here are the formulas I used to calculate each part:
$$
\text{The grand mean: } \overline{\overline{y}} = \frac{1}{n} \sum{\text{all data points}} \\
\text{The group mean: } \overline{y_i} = \frac{1}{n_i} \sum_{j=1}^{n_i}y_{ij} \\
\text{The between groups sum of squares: } \sum_{i=1}^{I} \sum_{j=1}^{n_i} \Big(\overline{y_i} - \overline{\overline{y}}\Big)^2 = \sum_{i=1}^{I} n_i \Big(\overline{y_i} - \overline{\overline{y}}\Big)^2 \\
\text{The within groups sum of squares: } \sum_{i=1}^{I}\sum_{j=1}^{n_i} (y_{ij} - \overline{y_i})^2
$$

**Part D:** Perform a **t hypothesis test** to determine if there is evidence supporting the claim that cinnamon bun Oreos have a higher mean rating than PB&J Oreos. Use the significance level you identified in **Part B**. Clearly state your null and alternative hypotheses, your conclusions, and show all work. Again, you may not use any canned t-test function.

**Answer D**
<br>For the following t-test I will be using the following hypothesis' and significance level
$$
H_0 : \overline{X}_\text{Cinnamon} - \overline{X}_\text{PB&J} = 0 \\
H_1 : \overline{X}_\text{Cinnamon} - \overline{X}_\text{PB&J} > 0 \\
\alpha = 0.1
$$

In [12]:
num = dfA['cinnamon bun']['average_across_raters'] - dfA['pb&j']['average_across_raters']
denom = np.sqrt(((np.std(dfA['cinnamon bun'][:-1]) ** 2)/ len(dfA['cinnamon bun'][:-1].dropna().values)) +
               ((np.std(dfA['pb&j'][:-1]) ** 2 / (len(dfA['pb&j'][:-1].dropna().values)))))
ts = num / denom
p_val = 1 - stats.t.cdf(ts, len(dfA['cinnamon bun'][:-1].dropna().values) + (len(dfA['pb&j'][:-1].dropna().values) - 2))
print('P-Value: {0:.3f}'.format(p_val))
print('T-Statistic: {0:.3f}'.format(ts))

P-Value: 0.016
T-Statistic: 2.962


Our calculated P-Value, $0.016$ is less than our significance $\alpha=0.1$ so we can conclude that at the 10% significance level there is enough evidence to reject the null hypothesis.

**Part E:** Do your results from Parts C and D agree with one another? If they agree, explain how they are in agreement in _words_. If they do not agree, explain why you think they do not agree.

_Hint: if they do not agree, consider carefully what assumptions we make in ANOVA and the t-test._

**Answer E**
<br>My results from parts (C) and (D) do not agree with one another. When doing a t-test we would like for our data to resemble some sort of normal distribution and that the variances are roughly the same between groups. In ANOVA we also would like our data to look normally distributed but are testing to see if the variances between groups are roughly equal. This difference in assumptions may account for the fact that these two tests give two different results. Another thing to take into account is that the t-test is only testing the means between two flavors of oreos while the ANOVA is testing over all the flavors.

<br>

---
<a id='p2'></a>
[Back to top](#top)

### [30 points] Problem 2: Multiple Linear Regression to Explain House Hauntings

<img src="https://s-media-cache-ak0.pinimg.com/originals/09/72/01/09720128cff5de4d4af038cd3fcf7f69.jpg" style="width: 300px;"/>

In an effort to control the skyrocketing prices of real estate in the Colorado Front Range, Governor Polis implemented a cutting edge new intervention. This new program oversaw the introduction of ghosts back into their natural ecosystem, after the ghost population seriously dwindled in recent decades due to overhaunting. However, an unfortunate miscalculation has led to haunted houses becoming a very serious problem in Colorado. Modern problems require modern solutions, so Governor Polis has hired you and the famous hedgehog data scientist/part-time ghostbuster Amy to determine what features of a house may be used to best predict a `haunted` score, related to the probability that a house with the given features is haunted (higher $\leftrightarrow$ more likely to be haunted).

You decide to use multiple linear regression to understand and predict what factors lead to increased haunted house hazard. You collected a data set from Haunted Zillow, the lesser-known database of haunted house prices and attributes. The data cover a variety of potential features, and you'll find this data in the file `houses.csv`. 

**Response**: 

- $\texttt{haunted}$: a haunting score, related to the probability that a house with the given features is haunted (higher $\leftrightarrow$ more likely to be haunted)

**Features**: 

- $\texttt{age}$: age of the house, in years
- $\texttt{area}$: square footage of interior of house
- $\texttt{bathrooms}$: number of bathrooms
- $\texttt{distance metro}$: distance to the nearest major metropolitan area (in miles)
- $\texttt{distance cemetery}$: distance to the nearest cemetery (in miles)
- $\texttt{cats}$: the number of cats within a one-block radius of the house
- $\texttt{howls}$: the number of wolf howls heard on an average night in the house's neighborhood
- $\texttt{clouds}$: what percentage of the sky was covered by clouds (fraction, 0-1)
- $\texttt{precipitation}$: amount of precipitation in the past 72 hours (inches)
- $\texttt{misery index}$: an economic indicator for how miserable the average United States citizen is, based on the unemployment rate and the inflation rate. More [here](https://www.stuffyoushouldknow.com/podcasts/whats-the-misery-index.htm) and [here](https://en.wikipedia.org/wiki/Misery_index_(economics)). Higher values correspond to more miserable citizens.
- $\texttt{ice cream sold}$: the number of units of ice cream sold at the farmer's market the week the house was most recently sold

**Part A**: Read the data from `houses.csv` into a Pandas DataFrame.  Note that since we will be doing a multiple linear regression we will need all of the features, so you should drop any row in the DataFrame that is missing data. 

In [2]:
df = pd.read_csv('houses.csv')
df = df.dropna()
df.head()

,age,area,bathrooms,distance metro,distance cemetery,cats,howls,clouds,precipitation,misery index,ice cream sold,haunted
0,65.06,2041,1.0,7.1,10.01,7,3,1.00,0.82,12.99,273,-0.596150
1,141.48,1564,0.0,7.4,4.07,5,5,1.00,0.99,16.77,184,-0.146465
2,7.58,1637,3.0,7.0,3.36,2,0,1.00,1.17,16.49,141,-0.303117
3,51.47,2021,2.0,7.9,3.43,6,8,0.13,0.92,8.28,146,0.339912
4,259.44,1642,1.0,7.5,3.19,4,1,1.00,1.73,5.90,178,0.724867


**Part B**: Perform the appropriate statistical test at the $\alpha = 0.01$ significance level to determine if _at least one_ of the features is related to the the response $y$.  Clearly describe your methodology and show all computations in Python. 

**Answer B**
<br>For the above statistical test we will use the following hypothesis
$$
H_0 : \beta_1 = \beta_2 = ... = \beta_n \\
H_1 : \beta_i = 0 \text{ for some }i
$$

In [3]:
#extract the features and response in respective X, Y variables
x, y = df.iloc[:,:-1], df['haunted']

x = sm.add_constant(x)
model = sm.OLS(y,x).fit()
model.summary()
#model.params

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                haunted   R-squared:                       0.948
Model:                            OLS   Adj. R-squared:                  0.938
Method:                 Least Squares   F-statistic:                     94.60
Date:                Fri, 03 May 2019   Prob (F-statistic):           1.97e-32
Time:                        10:53:09   Log-Likelihood:                 13.575
No. Observations:                  69   AIC:                            -3.151
Df Residuals:                      57   BIC:                             23.66
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.3480      0.323      1.078      0.285      -0.298       0.994
age                   0.0017      0.000      5.859      0.000       0.001       0.002
area                5.83e-05   6.16e-05      0.946      0.348   -6.51e-05       0.000
bathrooms            -0.0411      0.023     -1.806      0.076      -0.087       0.004
distance metro       -0.0142      0.032     -0.443      0.660      -0.079       0.050
distance cemetery    -0.1000      0.003    -30.867      0.000      -0.106      -0.093
cats                  0.0316      0.010      3.258      0.002       0.012       0.051
howls                 0.0056      0.011      0.494      0.623      -0.017       0.029
clouds                0.0902      0.123      0.734      0.466      -0.156       0.336
precipitation         0.0068      0.067      0.102      0.919      -0.128       0.141
misery index         -0.0082      0.006     -1.456      0.151      -0.020       0.003
ice cream sold       -0.0006      0.001     -1.274      0.208      -0.002       0.000
==============================================================================
Omnibus:                        1.629   Durbin-Watson:                   2.191
Prob(Omnibus):                  0.443   Jarque-Bera (JB):                1.480
Skew:                           0.230   Prob(JB):                        0.477
Kurtosis:                       2.450   Cond. No.                     2.55e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.55e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Because our model has an $F$-statistic of 94.60 and an associated p-value much less than our $\alpha = 0.01$ we can reject the null hypothesis stated above and conclude that at least one feature has some effect on the response, y.

**Part C**: Write a function `forward_select(df, resp_str, maxk)` that takes in the DataFrame, the name of the column corresponding to the response, and the maximum number of desired features, and returns a list of feature names corresponding to the `maxk` most important features via forward selection.  At each stage in forward selection you should add the feature whose inclusion in the model would result in the lowest sum of squared errors $(SSE)$. Use your function to determine the best $k=5$ features to include in the model. Clearly indicate which feature was added in each stage. 

**Note**: The point of this exercise is to see if you can implement **foward_select** yourself.  You may of course use canned routines like statmodels OLS, but you may not call any Python method that explicitly performs forward selection.

In [4]:
# the following function will run a manual forward-selection process
def forward_select_fstat(df, resp_str, maxk):
    dfA = df
    dfA = dfA.drop('haunted', axis=1)
    ans = []
    y = df[resp_str]
    while(len(ans) != maxk):
        to_add = ''
        mx = 0
        cols = list(dfA)
        for i in cols:
            ans.append(i)
            xcur = dfA[i]
            xcur = sm.add_constant(xcur)
            tstmod = sm.OLS(y, df[ans]).fit()
            fstat = tstmod.fvalue
            if(fstat > mx):
                to_add = i
                mx = fstat
            ans.pop()
        print("The feature added to our model was: {0}".format(to_add))
        ans.append(to_add)
        dfA = dfA.drop(ans[-1], axis=1)
        model = sm.OLS(y, df[ans]).fit()
    print("\n{}".format(model.summary()))
    return ans

feat = forward_select_fstat(df, 'haunted', 5);

The feature added to our model was: distance cemetery
The feature added to our model was: age
The feature added to our model was: cats
The feature added to our model was: area
The feature added to our model was: bathrooms

                            OLS Regression Results                            
Dep. Variable:                haunted   R-squared:                       0.944
Model:                            OLS   Adj. R-squared:                  0.940
Method:                 Least Squares   F-statistic:                     216.0
Date:                Fri, 03 May 2019   Prob (F-statistic):           1.12e-38
Time:                        10:53:10   Log-Likelihood:                 11.000
No. Observations:                  69   AIC:                            -12.00
Df Residuals:                      64   BIC:                           -0.8288
Df Model:                           5                                         
Covariance Type:            nonrobust                             

# PLEASE READ
<br>I realized way too late on Friday afternoon that I have completely screwed up part C. Instead of writing a forward select based on lowest SSE I went with the slides which said you can also look for the largest f-statistic and use that. I understand I have made a very stupid mistake and hope you take mercy on me.

**Part D**: Write down the multiple linear regression model, including estimated parameters, obtained by your forward selection process. 

**Answer D**
<br>Using my forward selection process (which very well may be terribly wrong) I obtained the following multiple linear regression model:
$$
y = -0.1003(x_4) + 0.0019(x_0) + 0.0332(x_5) + 0.0000976(x_1) - 0.0277(x_2)
$$

**Part E**: Perform the appropriate statistical test at the $\alpha = 0.05$ significance level to determine whether there is a statistically significant difference between the full model with all features and the reduced model obtained by forward selection in **Part D**. 

In [5]:
sse_red = 0
x_red = df[feat]
x_red = sm.add_constant(x_red)
model_red = sm.OLS(y, x_red).fit()

In [6]:
y_hat_red = model_red.params[0]
for _ in feat:
    y_hat_red += model_red.params[_]*x_red[_]
sse_red = np.sum((y-y_hat_red)**2)
print('SSE for the reduced model: {0:.3f}'.format(sse_red))

SSE for the reduced model: 2.917


In [7]:
sse_full = 0
y_hat_full = model.params[0]
for _ in list(x.columns[1:]):
    y_hat_full += model.params[_]*x[_]
sse_full = np.sum((y-y_hat_full) ** 2)
print('SSE for the full model: {0:.3f}'.format(sse_full))
num = (sse_red-sse_full)/(11-5)
denom = sse_full/(69-12)
fcomp = num/denom
pval = 1 - stats.f.cdf(fcomp, 11-5, 69-12)
print('The p-value for the comparison between models is: {0}'.format(pval))

SSE for the full model: 2.726
The p-value for the comparison between models is: 0.6755144140021556


**Answer E**
<br>We will use the following subset of features hypothesis test to figure out which model is "better".
$$
H_0 : \beta_{metro} = \beta_{howls} = \beta_{clouds} = \beta_{precip} = \beta_{misery} = \beta_{ice cream sold} = 0 \\
H_1 : \text{One of the above features is significant to the model}
$$
<br>Because our $\text{p-value} = 0.676$ is larger than our $\alpha = 0.05$ we can conclude that at the 5% significance level we do not have enough evidence to reject the null hypothesis.

**Part F**: Based on your conclusions in **Part E**, use the _better_ of the two models to predict the haunted-house hazard when the following features are observed: 

- $\texttt{age}$: 100 years
- $\texttt{area}$: 2200 square feet
- $\texttt{bathrooms}$: 3 bathrooms
- $\texttt{distance metro}$: 25 miles
- $\texttt{distance cemetery}$: 10 miles
- $\texttt{cats}$: 4 cats
- $\texttt{howls}$: 5 wolf howls/night
- $\texttt{clouds}$: 0.65 cloud cover
- $\texttt{precipitation}$: 0 inches
- $\texttt{misery index}$: 10
- $\texttt{ice cream sold}$: 125

**Answer F**
<br>My results in E conclude that the reduced model is a _better_ model to fit our data based on the difference in $R^2$ and $R_a^2$ values. The difference between those two values in my reduced model is closer to 0 than in the full model which suggests to me that the reduced model is a _better_ model.

In [8]:
pred_red = np.array([100, 2200, 3, 10, 4])
print("The prediction with the reduced model is: {0:.3f}".format(model_red.params[0] + np.sum(pred_red*model_red.params[1:])))

The prediction with the reduced model is: -6.033


**Part G:** Governor Polis dabbles a bit in the art of data science, as well as the science of data art. He tells you that the response (`haunted` score) that you and Amy predicted is actually the natural logarithm of the _odds_ that a house with the given features is haunted, where if $p$ is the probability that a house is haunted, then the odds are given by $$\text{odds} = \dfrac{p}{1-p}$$

What transformation must you make in order to turn your multiple regression model into a logistic regression model, to classify a house a haunted or not? Perform this simple transformation, then use a decision threshold of 0.5 to classify the house from **Part F** as haunted or not haunted. No new models should be fit here; use the same model that you used in Part F.

**Answer G**
<br>To turn our MLR model into a LogReg model we need to put our model into the sigmoid function 
$$sigm(z) = \frac{1}{1+e^{-(\beta_0 + \beta_1x_1 + \beta_2x_2 + ...)}}$$
<br>We can do that with the value found in part $F$ above and we obtain the following function
$$
sigm(-6.033) = \frac{1}{1+e^{6.033}} \\
sigm(-6.033) = \frac{1}{1+416.96} = \frac{1}{417.96} = 0.00239\\
$$

In [10]:
#Code to check the above result

def sigmoid(x):
    return 1/(1+np.exp(-x))

yhat_f = model_red.params[0] + np.sum(pred_red*model_red.params[1:])
logval = sigmoid(yhat_f)
print("Before LogReg: {0:.3f}\nAfter LogReg: {1:.5f}".format(yhat_f, logval))
if(logval > 0.5):
    print("Based on the decision threshold of 0.5, the house is haunted.")
else:
    print("Based on the decision threshold of 0.5, the house is NOT haunted.")

Before LogReg: -6.033
After LogReg: 0.00239
Based on the decision threshold of 0.5, the house is NOT haunted.


<br>

---
[Back to top](#top)
<a id='bot'></a>